In [2]:
import pandas as pd
import numpy as np

In [8]:
df_nonsplit = pd.read_csv("alignments.tsv", sep="\t")
df_nonsplit = df_nonsplit[(df_nonsplit["split"] == False) & df_nonsplit["query_cigar"].str.contains('S')]
df_nonsplit.to_csv("special_cases.csv")
pd.set_option('display.max_colwidth', None)
df_nonsplit = df_nonsplit[['split', 'AA_homology_seq', 'break_pos1', 'break_pos2', 'query_name', 'query_chrom', 'query_pos', 'query_end', 'query_orientation', 'query_cigar', 'query_aln_sub', 'query_aln_full']]
df_nonsplit.query_name = df_nonsplit.query_name.apply(lambda x: x.split(':')[-1])
df_nonsplit
nonsplit_matches = df_nonsplit[
    (df_nonsplit["break_pos1"] == df_nonsplit["query_pos"]) | 
    (df_nonsplit["break_pos1"] == df_nonsplit["query_end"]) | 
    (df_nonsplit["break_pos2"] == df_nonsplit["query_pos"]) | 
    (df_nonsplit["break_pos2"] == df_nonsplit["query_end"])
]

In [ ]:
bs_sv = set(df_nonsplit[df_nonsplit["AA_homology_seq"].isna()]['break_pos1']).union(set(df_nonsplit[df_nonsplit["AA_homology_seq"].isna()]['break_pos2'])).remove(94023408)
bs_sv
df2 = pd.read_csv("alignments.tsv", sep="\t")
thems_the_breaks = set([item for tup in set(list(zip(zip(df2.break_chrom1, df2.break_pos1), zip(df2.break_chrom2, df2.break_pos2)))) for item in tup if item[1] not in bs_sv])
thems_the_breaks
def check_break(row):
    for i in thems_the_breaks:
        if row['query_chrom'] == i[0] and row['query_pos'] <= i[1] and row['query_end'] >= i[1]:
            return i
    return np.nan
df2['breakpoint'] = df2.apply(check_break, axis=1)
df2.to_csv('with_breaks.csv')

In [ ]:
df_split = pd.read_csv("alignments.tsv", sep="\t")
df_split = df_split[df_split["split"] == True]
pd.set_option('display.max_colwidth', None)
df_split = df_split[['split', 'AA_homology_seq', 'break_pos1', 'break_pos2', 'query_name', 'query_chrom', 'query_pos', 'query_end', 'query_orientation', 'query_cigar', 'query_aln_sub', 'query_aln_full']]
df_split.query_name = df_split.query_name.apply(lambda x: x.split(':')[-1])
df_split
half_match = df_split[
    (df_split["break_pos1"] == df_split["query_pos"]) | 
    (df_split["break_pos1"] == df_split["query_end"]) | 
    (df_split["break_pos2"] == df_split["query_pos"]) | 
    (df_split["break_pos2"] == df_split["query_end"])
]
full_match = half_match[half_match.groupby("query_name")["query_name"].transform("size") > 1]
full_match

,split,AA_homology_seq,break_pos1,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full
0,True,GAGTG,133607147,23632742,92037,chr9,133607147,133607212,+,34S66M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA
1,True,GAGTG,133607147,23632742,92037,chr22,23632704,23632742,+,39M61H,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG
3,True,GAGTG,133607147,23632742,31831,chr22,23632682,23632742,-,61M39S,GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA
4,True,GAGTG,133607147,23632742,31831,chr9,133607147,133607190,-,56H44M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA
7,True,GAGTG,133607147,23632742,88045,chr9,133607147,133607177,-,69H31M,GAGTGGGTTTTATCAGCTTCCATACCCAAAC,GAGTGGGTTTTATCAGCTTCCATACCCAAAC
8,True,GAGTG,133607147,23632742,88045,chr22,23632669,23632742,-,74M26S,GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAAC
9,True,GAGTG,133607147,23632742,93025,chr9,133607147,133607217,-,29S71M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA
10,True,GAGTG,133607147,23632742,93025,chr22,23632709,23632742,-,34M66H,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG
64,True,AT,134155524,108661411,29524,chr9,134155484,134155524,-,41M59H,TCTCAGTCTGCAGGAGAAGGGACCAAGAAGCATCAGTCCAT,TCTCAGTCTGCAGGAGAAGGGACCAAGAAGCATCAGTCCAT
65,True,AT,134155524,108661411,29524,chr13,108661351,108661411,+,61M39S,ATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,ATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCTTGGTCCCTTCTCCTGCAGACTGAGA


In [120]:
new_awesomesauce_df =pd.concat([nonsplit_matches, full_match], axis=0).sort_values(["AA_homology_seq", "split"])
new_awesomesauce_df["break_start"] = (new_awesomesauce_df["break_pos1"] == new_awesomesauce_df["query_pos"]) | (new_awesomesauce_df["break_pos2"] == new_awesomesauce_df["query_pos"])
new_awesomesauce_df

,split,AA_homology_seq,break_pos1,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full,break_start
237,True,AA,92474011,92475758,57596,chr13,92473942,92474011,-,70M30S,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTT,False
238,True,AA,92474011,92475758,57596,chr13,92475727,92475758,+,32M68H,AAATTAATGCTCCCACTTCAACTTAAAGAATT,AAATTAATGCTCCCACTTCAACTTAAAGAATT,False
241,True,AA,92474011,92475758,83594,chr13,92475722,92475758,-,37M63H,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,False
242,True,AA,92474011,92475758,83594,chr13,92473947,92474011,+,65M35S,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTTACTTT,False
114,True,AAC,81088033,81469935,78137,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True
115,True,AAC,81088033,81469935,78137,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True
118,True,AAC,81088033,81469935,60717,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True
119,True,AAC,81088033,81469935,60717,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True
86,False,AT,134155524,108661411,28031,chr9,134155451,134155524,-,74M26S,TCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCAT,TCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCATTATAATTTTCAAGACAAAGCAGGAAG,False
90,False,AT,134155524,108661411,44926,chr9,134155445,134155524,+,80M20S,ACCTGCTCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCAT,ACCTGCTCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCATTATAATTTTCAAGACAAAGC,False


In [ ]:
cooked_df = pd.read_csv("alignments.tsv", sep="\t")
cooked_df = cooked_df[
    (
        (cooked_df['query_chrom'] != cooked_df['break_chrom1']) &
        (cooked_df['query_chrom'] != cooked_df['break_chrom2'])
    ) | 
    ((
        (cooked_df['break_pos1'] > cooked_df['query_end']) |
        (cooked_df['break_pos1'] < cooked_df['query_pos'])
    ) &
    (
        (cooked_df['break_pos2'] > cooked_df['query_end']) |
        (cooked_df['break_pos2'] < cooked_df['query_pos'])
    ))
]
cooked_df = cooked_df.loc[cooked_df['query_cigar'].str.contains('S') | cooked_df['query_cigar'].str.contains('H')].drop(["break_sv_type", "break_read_support", "break_features", "query_aln_full"], axis=1)
cooked_df.to_csv("minor_errors.tsv", sep="\t")

In [ ]:
def process_row(row):
    if ((row["break_pos1"] == row["query_pos"] or row["break_pos1"] == row["query_end"]) and not row["break_start"]) or ((row["break_pos2"] == row["query_pos"] or row["break_pos2"] == row["query_end"]) and row["break_start"]):
        return row["query_aln_sub"][::-1]
    return row["query_aln_sub"]

new_awesomesauce_df["new_awesomesauce_query"] = new_awesomesauce_df.apply(process_row, axis=1)
new_awesomesauce_df

,split,AA_homology_seq,break_pos1,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full,break_start,homology_cut,wrong_dog,new_awesomesauce_query
237,True,AA,92474011,92475758,57596,chr13,92473942,92474011,-,70M30S,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTT,False,CATCAAA,False,AAACTACTAATCAATCGTAGGTAATTATATTATTAGTATACTTATCTAATCTTCTTTAGTGTAATGCAGA
238,True,AA,92474011,92475758,57596,chr13,92475727,92475758,+,32M68H,AAATTAATGCTCCCACTTCAACTTAAAGAATT,AAATTAATGCTCCCACTTCAACTTAAAGAATT,False,AAGAATT,False,AAATTAATGCTCCCACTTCAACTTAAAGAATT
241,True,AA,92474011,92475758,83594,chr13,92475722,92475758,-,37M63H,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,False,AAGAATT,False,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT
242,True,AA,92474011,92475758,83594,chr13,92473947,92474011,+,65M35S,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTTACTTT,False,CATCAAA,False,AAACTACTAATCAATCGTAGGTAATTATATTATTAGTATACTTATCTAATCTTCTTTAGTGTAAT
114,True,AAC,81088033,81469935,78137,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True,AACTTTT,False,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG
115,True,AAC,81088033,81469935,78137,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True,GTTTCTT,True,CAATTACTTTTTTTGTCTTCTTATTTCTGGTTTCTTTG
118,True,AAC,81088033,81469935,60717,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True,GTTTCTT,True,CAATTACTTTTTTTGTCTTCTTATTTCTGGTTTCTTTG
119,True,AAC,81088033,81469935,60717,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True,AACTTTT,False,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG
86,False,AT,134155524,108661411,28031,chr9,134155451,134155524,-,74M26S,TCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCAT,TCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCATTATAATTTTCAAGACAAAGCAGGAAG,False,AGTCCAT,False,TACCTGACTACGAAGGACCAGGGAAGAGGACGTCTGACTCTACTGTAGGACACGTCCCCACTACAGTTCCCCCT
90,False,AT,134155524,108661411,44926,chr9,134155445,134155524,+,80M20S,ACCTGCTCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCAT,ACCTGCTCCCCCTTGACATCACCCCTGCACAGGATGTCATCTCAGTCTGCAGGAGAAGGGACCAGGAAGCATCAGTCCATTATAATTTTCAAGACAAAGC,False,AGTCCAT,False,TACCTGACTACGAAGGACCAGGGAAGAGGACGTCTGACTCTACTGTAGGACACGTCCCCACTACAGTTCCCCCTCGTCCA


In [188]:
first = [
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGG",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATA",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGGTTTCTTTGATCCTAG",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAAC",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA",
]
last = [
    'CTCCACCCAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
           'CAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
            'AGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                                                             'TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                                       'GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                          'GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                                                                  'ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
]

def mini_homology_inator(str_list, dir):
    str_list = [i[::dir] for i in str_list]
    slice = 0
    for i in range(len(max(str_list, key=len))):
        temp_list = [i[:slice] for i in str_list]
        if not len(set(temp_list)) <= 1:
            break
        slice += 1
    return str_list[0][:slice - 1][::dir]

def homology_inator(first, last):
    first_str = mini_homology_inator(first, 1)
    last_str = mini_homology_inator(last, -1)
    longest_str = ''
    for i in range(min(len(first_str), len(last_str))):
        if first_str[:i] == last_str[-i:]:
            longest_str = first_str[:i]
    print(longest_str)
    return longest_str

homology_inator(first, last)

GAGTG


'GAGTG'

In [192]:
new_awesomesauce_df["first"] = new_awesomesauce_df.apply(lambda row: row["break_pos1"] == row["query_pos"] or row["break_pos1"] == row["query_end"], axis=1)
grouped_queries = new_awesomesauce_df.groupby(["break_pos1", "first"])["new_awesomesauce_query"].agg(list)
grouped_queries = grouped_queries.reset_index().pivot(index="break_pos1", columns="first", values="new_awesomesauce_query")
grouped_queries.columns = ["last", "first"]
grouped_queries['homology'] = grouped_queries.apply(lambda row: homology_inator(row["first"], row["last"]), axis=1)
grouped_queries

T

GGT



GAGTG
T


,last,first,homology
break_pos1,,,
18953123,[AATTGATTGAGTCGTGTTTGAGACTCTGGGTT],[TCCATCTGGGAAACGGAACAAACCTGCATGCCGCATGGATCATGGTGGCCCAGAGTGGGGTGCCCAGG],T
81088033,"[CAATTACTTTTTTTGTCTTCTTATTTCTGGTTTCTTTG, CAATTACTTTTTTTGTCTTCTTATTTCTGGTTTCTTTG]","[AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG, AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG]",
81109414,"[TTTATCTTAATATTAGTTCTACCTGGTTTTTATTTCAATTTTTTGACTCGAATTCTAAAACGGGT, GTTTTTATTTCAATTTTTTGACTCGAATTCTAAAACGGGT]","[GGTCTGTATACTCCATTTTAATTAAGTCAGTCACTGTCGATAATACATTGGTTATGTAACGTGACACCTCTATTTGTGTATCCGTG, GGTCTGTATACTCCATTTTAATTAAGTCAGTCACTGTCGATAATACATTGGTTATGTAACGTGACACGTCTATTTG, GGTCTGTATACTCCATTTTAATTAAGTCAGTCACTGTCGATAATACATTGGTTATGTAACGTGACACCTCTATT, GGTCTGTATACTCCATTTTAATTAAGTCAGTCACTGTCGATAATACATTGGTTATGTAACGTGACACCTCTATTTGTGTAT, GGTCTGTATACTCCATTTTAATTAAGTCAGTCTCTGTCGATAATACATTGGTTATGTAACGTGACACCTCTATTTGTGT, GGTCTGTATACTCCATTTTAATTAAGTCAGTCACTGTC, GGTCTGTATACTCCATTTTAATTAAGTCAGTCACTGTCGATAATACATTGGTTATGTAACGTG]",GGT
81470036,"[ATGCCTGAATAATATTGATATTAGTTATTTGTAACTTGTATCTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT, CTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT]",[AAAATAAAGTCTGAGTATACCTTGTAGGCACACATACACATATTTATCAAATA],
92474011,"[AAATTAATGCTCCCACTTCAACTTAAAGAATT, AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT]","[AAACTACTAATCAATCGTAGGTAATTATATTATTAGTATACTTATCTAATCTTCTTTAGTGTAATGCAGA, AAACTACTAATCAATCGTAGGTAATTATATTATTAGTATACTTATCTAATCTTCTTTAGTGTAAT]",
94023408,[CCTCTCTGTGTCTCCTACGAATCCCTTAAATCCTATCTA],"[TTTTACGTATACAGTTCTTGATACAATGTAAAGTTGGATCAAAAGTTTTAGTCTTATGTTGTAGACAGTGTTGGTAGGTGTATCGACACCTGATACAA, TTTTACGTATACAGTTCTTGATACAATGTAAAGTTGGATCAAAAGTTTTAGTCTTATGTTGTAGACAGTGTTGGTAGGTGTATCGAC, TTTTACGTATACAGTTCTTGATACAATGTAAAGTTGGATCAAAAGTTTTAGTCTTATGTTG]",
133607147,"[CTCCACCCAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, CAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, AGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG]","[GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGG, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATA, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGGTTTCTTTGATCCTAG, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA, GAGTGGGTTTTATCAGCTTCCATACCCAAAC, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA]",GAGTG
134155524,"[TAAACATTCAAGTTTTATGACTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, AGTTTTATGACTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, ATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, TATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, GAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT]","[TACCTGACTACGAAGGACCAGGGAAGAGGACGTCTGACTCTACTGTAGGACACGTCCCCACTACAGTTCCCCCT, TACCTGACTACGAAGGACCAGGGAAGAGGACGTCTGACTCTACTGTAGGACACGTCCCCACTACAGTTCCCCCTCGTCCA, TACCTGACTACGAAGAACCAGGGAAGAGGACGTCTGACTCT, TACCTGACTACGAAGGACCAGGGAAGAGGACGTCTGACTC, TACCTGACTACGAAGGACCAGGGAAGAGGACG]",T


In [ ]:
new_awesomesauce_df["homology_cut"] = new_awesomesauce_df.apply(lambda x: x['query_aln_sub'][:7] if x["break_start"] else x['query_aln_sub'][-7:], axis=1)
new_awesomesauce_df["wrong_dog"] = new_awesomesauce_df.apply(lambda x: str(x['AA_homology_seq']) not in x['homology_cut'], axis=1)
new_awesomesauce_df.loc[new_awesomesauce_df["wrong_dog"] == True]

,split,AA_homology_seq,break_pos1,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full,break_start,homology_cut,wrong_dog,new_awesomesauce_query
115,True,AAC,81088033,81469935,78137,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True,GTTTCTT,True,CAATTACTTTTTTTGTCTTCTTATTTCTGGTTTCTTTG
118,True,AAC,81088033,81469935,60717,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True,GTTTCTT,True,CAATTACTTTTTTTGTCTTCTTATTTCTGGTTTCTTTG
218,False,ATAAAA,81470036,81470311,69813,chr13,81470218,81470311,+,94M6S,ATGCCTGAATAATATTGATATTAGTTATTTGTAACTTGTATCTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT,ATGCCTGAATAATATTGATATTAGTTATTTGTAACTTGTATCTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTATTTCAGA,False,CTTTTAT,True,ATGCCTGAATAATATTGATATTAGTTATTTGTAACTTGTATCTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT
192,True,ATAAAA,81470036,81470311,15030,chr13,81470259,81470311,-,53M47H,CTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT,CTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT,False,CTTTTAT,True,CTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT
284,False,NaN,94023408,108500973,87256,chr13,94023311,94023408,+,98M2S,AACATAGTCCACAGCTATGTGGATGGTTGTGACAGATGTTGTATTCTGATTTTGAAAACTAGGTTGAAATGTAACATAGTTCTTGACATATGCATTTT,AACATAGTCCACAGCTATGTGGATGGTTGTGACAGATGTTGTATTCTGATTTTGAAAACTAGGTTGAAATGTAACATAGTTCTTGACATATGCATTTTAT,False,GCATTTT,True,TTTTACGTATACAGTTCTTGATACAATGTAAAGTTGGATCAAAAGTTTTAGTCTTATGTTGTAGACAGTGTTGGTAGGTGTATCGACACCTGATACAA
285,False,NaN,94023408,108500973,93655,chr13,94023322,94023408,+,87M13S,CAGCTATGTGGATGGTTGTGACAGATGTTGTATTCTGATTTTGAAAACTAGGTTGAAATGTAACATAGTTCTTGACATATGCATTTT,CAGCTATGTGGATGGTTGTGACAGATGTTGTATTCTGATTTTGAAAACTAGGTTGAAATGTAACATAGTTCTTGACATATGCATTTTATCTATCCTAAAT,False,GCATTTT,True,TTTTACGTATACAGTTCTTGATACAATGTAAAGTTGGATCAAAAGTTTTAGTCTTATGTTGTAGACAGTGTTGGTAGGTGTATCGAC
268,True,NaN,94023408,108500973,62556,chr13,108500973,108501011,+,61H39M,ATCTATCCTAAATTCCCTAAGCATCCTCTGTGTCTCTCC,ATCTATCCTAAATTCCCTAAGCATCCTCTGTGTCTCTCC,True,ATCTATC,True,CCTCTCTGTGTCTCCTACGAATCCCTTAAATCCTATCTA
269,True,NaN,94023408,108500973,62556,chr13,94023348,94023408,+,61M39S,GTTGTATTCTGATTTTGAAAACTAGGTTGAAATGTAACATAGTTCTTGACATATGCATTTT,GTTGTATTCTGATTTTGAAAACTAGGTTGAAATGTAACATAGTTCTTGACATATGCATTTTATCTATCCTAAATTCCCTAAGCATCCTCTGTGTCTCTCC,False,GCATTTT,True,TTTTACGTATACAGTTCTTGATACAATGTAAAGTTGGATCAAAAGTTTTAGTCTTATGTTG
300,True,NaN,18953123,22935374,14734,chr22,22935374,22935405,+,68H32M,TTGGGTCTCAGAGTTTGTGCTGAGTTAGTTAA,TTGGGTCTCAGAGTTTGTGCTGAGTTAGTTAA,True,TTGGGTC,True,AATTGATTGAGTCGTGTTTGAGACTCTGGGTT
301,True,NaN,18953123,22935374,14734,chr22,18953123,18953190,-,32S68M,TCCATCTGGGAAACGGAACAAACCTGCATGCCGCATGGATCATGGTGGCCCAGAGTGGGGTGCCCAGG,TTAACTAACTCAGCACAAACTCTGAGACCCAATCCATCTGGGAAACGGAACAAACCTGCATGCCGCATGGATCATGGTGGCCCAGAGTGGGGTGCCCAGG,True,TCCATCT,True,TCCATCTGGGAAACGGAACAAACCTGCATGCCGCATGGATCATGGTGGCCCAGAGTGGGGTGCCCAGG
